In [ ]:
import pandas as pd



In [ ]:
!wget https://media.githubusercontent.com/media/TheArchitect2000/AI-CloudSec-System/refs/heads/team11/models/datasets/team_11_TON_IoT_unsw_edu_au_cleaned.csv


In [ ]:
df=pd.read_csv('team_11_TON_IoT_unsw_edu_au_cleaned.csv')

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.info

In [ ]:
df.info()

In [ ]:
import pandas as pd

# Assuming df is your DataFrame

# Count number of 0s in each column
zero_counts_per_column = (df == 0).sum(axis=0)

print(zero_counts_per_column)


In [ ]:
!wget https://media.githubusercontent.com/media/TheArchitect2000/AI-CloudSec-System/refs/heads/team11/models/datasets/team_11_BotNeTIoT_test.csv
!wget https://media.githubusercontent.com/media/TheArchitect2000/AI-CloudSec-System/refs/heads/team11/models/datasets/team_11_BotNeTIoT_train.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

# -------------------
# Load train and test data
# -------------------
train_df = pd.read_csv("team_11_BotNeTIoT_train.csv")
test_df = pd.read_csv("team_11_BotNeTIoT_test.csv.1")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Columns:", train_df.columns)

# -------------------
# Preprocessing
# -------------------

# Drop irrelevant columns (IDs, IPs, timestamps if present)
drop_cols = ["Unnamed: 0", "ts", "src_ip", "dst_ip"]
train_df = train_df.drop(columns=[c for c in drop_cols if c in train_df.columns])
test_df = test_df.drop(columns=[c for c in drop_cols if c in test_df.columns])

# Convert object columns (like src_bytes) to numeric
for col in train_df.columns:
    if train_df[col].dtype == "object":
        train_df[col] = pd.to_numeric(train_df[col], errors="coerce")
        test_df[col] = pd.to_numeric(test_df[col], errors="coerce")

# Fill missing values with 0
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

# -------------------
# Target column (multi-class)
# -------------------
# Change "type" → if your labels are in "label" instead
target_col = "label"

# Encode target labels
le = LabelEncoder()
# Combine train and test labels for fitting the encoder
all_labels = pd.concat([train_df[target_col], test_df[target_col]], axis=0)
le.fit(all_labels)

train_df[target_col] = le.transform(train_df[target_col])
test_df[target_col] = le.transform(test_df[target_col])

# Split features and target
X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

X_test = test_df.drop(columns=[target_col])
y_test = test_df[target_col]

# -------------------
# Scale numeric features
# -------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -------------------
# Train Model
# -------------------
model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# -------------------
# Evaluation
# -------------------
y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
print("\n🔢 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
!wget https://media.githubusercontent.com/media/TheArchitect2000/AI-CloudSec-System/refs/heads/team11/models/datasets/traffic_merged_cleaned_strict_noconstant.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ----------------------
# Load dataset
# ----------------------
df = pd.read_csv("traffic_merged_cleaned_strict_noconstant.csv")

print("Dataset shape:", df.shape)
print("Columns:", df.columns)

# ----------------------
# Features & Target
# ----------------------
target_col = "label"

X = df.drop(columns=[target_col])
y = df[target_col]

# ----------------------
# Handle categorical columns if present
# ----------------------
# Convert non-numeric columns into numeric (Label Encoding)
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = pd.factorize(X[col])[0]

# ----------------------
# Train-test split
# ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----------------------
# Feature scaling
# ----------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ----------------------
# Random Forest Model
# ----------------------
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"   # handles imbalance if attacks >> benign
)
rf_model.fit(X_train, y_train)

# ----------------------
# Predictions & Evaluation
# ----------------------
y_pred = rf_model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🔢 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
df

In [ ]:
df.info()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ----------------------
# Load dataset
# ----------------------
df = pd.read_csv("traffic_merged_cleaned_strict_noconstant.csv")

print("Dataset shape:", df.shape)
print("Columns:", df.columns)

# ----------------------
# Features & Target
# ----------------------
target_col = "Label"

X = df.drop(columns=[target_col])
y = df[target_col]

# Encode target labels (e.g., BENIGN -> 0, ATTACK -> 1)
le = LabelEncoder()
y = le.fit_transform(y)

# ----------------------
# Drop identifiers (not useful for prediction)
drop_cols = ["Flow ID", "Source IP", "Destination IP", "Timestamp"]
X = X.drop(columns=drop_cols, errors="ignore")

# Encode non-numeric columns in X (e.g., Protocol if categorical)
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = pd.factorize(X[col])[0]

# ----------------------
# Train-test split
# ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----------------------
# Feature scaling
# ----------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ----------------------
# Random Forest Model
# ----------------------
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)
rf_model.fit(X_train, y_train)

# ----------------------
# Predictions & Evaluation
# ----------------------
y_pred = rf_model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
print("\n🔢 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# ----------------------
# Load dataset (local file)
# ----------------------
df = pd.read_csv("traffic_merged_cleaned_strict_noconstant.csv")

# Inspect dataset
print("Columns:", df.columns)
print("Shape:", df.shape)

# ----------------------
# Features & Target
# ----------------------
target_col = "Label"

if target_col not in df.columns:
    raise ValueError(f"Target column '{target_col}' not found. Available: {df.columns}")

X = df.drop(columns=[target_col])
y = df[target_col]

# ----------------------
# Train-test split (NO stratify to avoid error)
# ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ----------------------
# Train Random Forest
# ----------------------
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# ----------------------
# Evaluate
# ----------------------
y_pred = rf.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# ----------------------
# Load dataset
# ----------------------
df = pd.read_csv("traffic_merged_cleaned_strict_noconstant.csv")

# ----------------------
# Target & Features
# ----------------------
target_col = "Label"  # capital L
X = df.drop(columns=[target_col])
y = df[target_col]

# ----------------------
# Drop identifier columns
# ----------------------
X = X.drop(columns=["Flow ID", "Source IP", "Destination IP", "Timestamp"], errors="ignore")

# ----------------------
# Convert object columns to numeric
# ----------------------
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = pd.factorize(X[col])[0]

# ----------------------
# Train-test split
# ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ----------------------
# Train Random Forest
# ----------------------
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# ----------------------
# Predictions & Evaluation
# ----------------------
y_pred = rf.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ----------------------
# Confusion Matrix
# ----------------------
cm = confusion_matrix(y_test, y_pred, labels=rf.classes_)
plt.figure(figsize=(12,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
